##### Code to preprocess / Organize and clean ProviApt LOG Data

In [2]:
import numpy as np
import pandas as pd
import os
import re
from datetime import date

In [3]:
import warnings    # to avoid warning during executions
warnings.filterwarnings("ignore")

In [24]:
# append a line to previous line if its not starting in stanard format
with open('..\..\dataExport\LOGdata\log_2022_4_20.csv', 'r+', encoding="utf-8") as file:
    text = str();
    for line in file:
        if line[0:3] == "202":
            text = text + '\n';
        text = text + line.strip();
    file.seek(0);
    file.write(text);

In [112]:
file_name = '..\..\dataExport\LOGdata\log_2022_4_20.csv'
log_rawDF = pd.read_csv(file_name, sep='*')

In [113]:
log_rawDF.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:0:13,1618916413,613129,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
1,2021-4-20 13:0:57,1618916457,9327,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
2,2021-4-20 13:1:38,1618916498,812633,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
3,2021-4-20 13:2:20,1618916540,214501,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
4,2021-4-20 13:3:3,1618916583,212528,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN


In [114]:
print(log_rawDF.shape)

(865668, 9)


In [115]:
print(log_rawDF.isnull().values.any())
print(log_rawDF.isnull().sum().sum())

True
824890


In [116]:
log_rawDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865668 entries, 0 to 865667
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Date_Time  865668 non-null  object
 1   Time       865668 non-null  int64 
 2   MicroSec   865668 non-null  int64 
 3   Type       865668 non-null  int64 
 4   Unit       865668 non-null  object
 5   Parameter  864368 non-null  object
 6   Message    865668 non-null  object
 7   Value1     411877 non-null  object
 8   Value2     495869 non-null  object
dtypes: int64(3), object(6)
memory usage: 59.4+ MB


In [117]:
log_rawDF['Parameter'].value_counts()

T10                 203861
Arduino             102024
feeding              80955
calculations         78184
AutoTemplateSync     49465
                     ...  
waste                    2
T41                      2
T44                      2
T45                      2
V42                      2
Name: Parameter, Length: 162, dtype: int64

In [118]:
log_rawDF['Parameter'].nunique()

162

In [119]:
log_parameters = log_rawDF['Parameter'].unique()
print(log_parameters)
print(type(log_parameters))

['Arduino' 'program' 'T61' 'water' 'T10' 'Reactor' 'T71' 'pneumatic'
 'autoSyncMonitor' 'T42A' 'harvest' 'calculations' 'P1L' 'feeding' 'T64'
 'T42' 'T53' 'T51' 'T73' 'T74' 'T65' 'T63' 'T72' 'T52' 'T54' 'PT1_V4'
 'T55' 'T43' 'LEDlighting' 'RM29' 'RM6' 'V74' nan 'RM12' 'RM13' 'RM14'
 'RM15' 'RM16' 'RM17' 'RM19' 'RM2' 'RM21' 'RM22' 'RM23' 'T41A' 'T43A'
 'RM35' 'RM4' 'RM5' 'RM7' 'P42' 'RM1' 'RM30' 'Arduino_QG' 'Arduino_pH'
 'V3' 'P1R' 'P2R' 'CO2_V2' 'R1' 'PTdiff_K2' 'L1W' 'Arduino_TT1'
 'Arduino_LT1' 'Arduino1_LT52' 'Arduino2_LT72' 'Arduino1_LT41'
 'Arduino2_LT61' 'AutoLink' 'P43' 'V42' 'P10' 'V11' 'V8' 'LT1' 'RM3'
 'RM11' 'RM8' 'M1' 'P2L' 'V61' 'P11' 'Recipes' 'Products' 'L1' 'V6' 'V1'
 'V72' 'chiller' 'T62' 'V71' 'V62' 'P12' 'V43' 'L1FR' 'L1R' 'L1B' 'LT10'
 'M10' 'V63' 'RM18' 'Arduino1' 'V52' 'V64' 'PT10' 'TT0' 'waste' 'V51'
 'RM20' 'RM33' 'AutoTemplateSync' 'Arduino2_LT73' 'Arduino_CO2'
 'Arduino2_LT63' 'Arduino2_LT65' 'Arduino1_LT53' 'Arduino2_LT64'
 'Arduino2_LT75' 'Arduino1_LT54' 'A

In [122]:
log_type = log_rawDF['Type'].unique()
print(log_type)

[4 3 2 5 6 1]


In [123]:
log_rawDF['Type'].value_counts()

2    516084
1    202503
4     67915
6     49397
5     18991
3     10778
Name: Type, dtype: int64

In [124]:
log_rawDF['Type'].isna().sum()

0

In [125]:
log_rawDF[log_rawDF['Type'].isnull()].index.tolist()

[]

In [126]:
log_unit = log_rawDF['Unit'].unique()
print(log_unit)
print(log_rawDF['Unit'].nunique())

['FPC11_FPP11' 'FPC14' 'FFU' 'FPC22' 'FPC21_AUT21' 'FPC21' 'FPC23' 'FPC13'
 'FPC24' 'FPC31' 'FPC23_FPP23' 'FPC41' 'FPC42' 'FPC11' 'Product'
 '&lt;EVAL&gt;' 'FPC21_FPP21' 'FPC13_FPP13' 'FPC24_FPP24' 'FFP_FFP'
 'FPC31_FPP31' 'FPC22_FPP22' 'FPC12' 'FPP11' 'FPC14_FPP14' 'FPP21' 'FPP13'
 'FPP14' 'FPP' 'FPC12_FPP' 'FPP31' 'docProduction' 'Dashboard' 'FFP_AUTFU'
 'FPC13_AUT13' 'FPC22_AUT22' 'FPC11_AUT11' 'FPC12_AUT' 'FPC31_AUT31'
 'FPC24_AUT24' 'FPC23_AUT23' 'FFU_harvestTank_DoseRequests'
 'FFU_T10_RecipeRequests' 'scheduler' 'ProductionTanks'
 'harvestTank-TotalTime' 'water-TotalTime' 'RM36-TotalTime'
 'AllComponentTanks' 'T10-TotalTime' 'cntrTankDetail' 'T71-TotalTime'
 'RM14-TotalTime' 'RM11-TotalTime' 'Reactor-TotalTime' 'RM38-TotalTime'
 'T64-TotalTime' 'RM31-TotalTime' 'RM3-TotalTime'
 'docPoduction-FF-TotalTime' 'RM37-TotalTime' 'T43A-TotalTime'
 'RM6-TotalTime' 'RM23-TotalTime' 'RM1-TotalTime' 'ComponentTanks'
 'RawMaterialTanks' 'FPC14_AUT14' 'dashboard']
69


In [127]:
print(log_rawDF['Type'].isna().sum())
print(log_rawDF['Unit'].isna().sum())
print(log_rawDF['Parameter'].isna().sum())

0
0
1300


In [128]:
log_rawDF = log_rawDF.dropna(subset=['Parameter'])
log_rawDF

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:0:13,1618916413,613129,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
1,2021-4-20 13:0:57,1618916457,9327,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
2,2021-4-20 13:1:38,1618916498,812633,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
3,2021-4-20 13:2:20,1618916540,214501,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
4,2021-4-20 13:3:3,1618916583,212528,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
...,...,...,...,...,...,...,...,...,...
865663,2022-4-20 13:0:24,1650452424,941081,1,FPC23_FPP23,Arduino,CO2 error value changed from 0.13 to 997,NaN,NaN
865664,2022-4-20 13:0:26,1650452426,155485,1,FPC23_AUT23,autoSyncMonitor,master of autoTemplateSync not found =&gt; no ...,NaN,NaN
865665,2022-4-20 13:0:26,1650452426,159772,1,FPC23_AUT23,autoSyncMonitor,Error!check autoTemplateSync template failed!,NaN,NaN
865666,2022-4-20 13:0:26,1650452426,164723,1,FPC23_AUT23,autoSyncMonitor,Timing exceeded warning level 11 &gt; 10,NaN,NaN


In [129]:
print(log_rawDF['Type'].isna().sum())
print(log_rawDF['Unit'].isna().sum())
print(log_rawDF['Parameter'].isna().sum())

0
0
0


In [130]:
log_rawDF.shape

(864368, 9)

In [131]:
log_unit = log_rawDF['Unit'].unique()
print(log_unit)
print(log_rawDF['Unit'].nunique())

['FPC11_FPP11' 'FPC14' 'FFU' 'FPC22' 'FPC21_AUT21' 'FPC21' 'FPC23' 'FPC13'
 'FPC24' 'FPC31' 'FPC23_FPP23' 'FPC41' 'FPC42' 'FPC11' 'Product'
 'FPC21_FPP21' 'FPC13_FPP13' 'FPC24_FPP24' 'FFP_FFP' 'FPC31_FPP31'
 'FPC22_FPP22' 'FPC12' 'FPP11' 'FPC14_FPP14' 'FPP21' 'FPP13' 'FPP14' 'FPP'
 'FPC12_FPP' 'FPP31' 'FFP_AUTFU' 'FPC13_AUT13' 'FPC22_AUT22' 'FPC11_AUT11'
 'FPC12_AUT' 'FPC31_AUT31' 'FPC24_AUT24' 'FPC23_AUT23' 'scheduler'
 'FPC14_AUT14']
40


In [132]:
log_parameter = log_rawDF['Parameter'].unique()
print(log_parameter)
print(log_rawDF['Parameter'].nunique())

['Arduino' 'program' 'T61' 'water' 'T10' 'Reactor' 'T71' 'pneumatic'
 'autoSyncMonitor' 'T42A' 'harvest' 'calculations' 'P1L' 'feeding' 'T64'
 'T42' 'T53' 'T51' 'T73' 'T74' 'T65' 'T63' 'T72' 'T52' 'T54' 'PT1_V4'
 'T55' 'T43' 'LEDlighting' 'RM29' 'RM6' 'V74' 'RM12' 'RM13' 'RM14' 'RM15'
 'RM16' 'RM17' 'RM19' 'RM2' 'RM21' 'RM22' 'RM23' 'T41A' 'T43A' 'RM35'
 'RM4' 'RM5' 'RM7' 'P42' 'RM1' 'RM30' 'Arduino_QG' 'Arduino_pH' 'V3' 'P1R'
 'P2R' 'CO2_V2' 'R1' 'PTdiff_K2' 'L1W' 'Arduino_TT1' 'Arduino_LT1'
 'Arduino1_LT52' 'Arduino2_LT72' 'Arduino1_LT41' 'Arduino2_LT61'
 'AutoLink' 'P43' 'V42' 'P10' 'V11' 'V8' 'LT1' 'RM3' 'RM11' 'RM8' 'M1'
 'P2L' 'V61' 'P11' 'Recipes' 'Products' 'L1' 'V6' 'V1' 'V72' 'chiller'
 'T62' 'V71' 'V62' 'P12' 'V43' 'L1FR' 'L1R' 'L1B' 'LT10' 'M10' 'V63'
 'RM18' 'Arduino1' 'V52' 'V64' 'PT10' 'TT0' 'waste' 'V51' 'RM20' 'RM33'
 'AutoTemplateSync' 'Arduino2_LT73' 'Arduino_CO2' 'Arduino2_LT63'
 'Arduino2_LT65' 'Arduino1_LT53' 'Arduino2_LT64' 'Arduino2_LT75'
 'Arduino1_LT54' 'Ardui

In [133]:
(log_rawDF['Unit']=='FPC11').value_counts()

False    847066
True      17302
Name: Unit, dtype: int64

In [134]:
log_rawDF.isnull().sum()

Date_Time         0
Time              0
MicroSec          0
Type              0
Unit              0
Parameter         0
Message           0
Value1       453017
Value2       368853
dtype: int64

In [135]:
log_rawDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 864368 entries, 0 to 865667
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Date_Time  864368 non-null  object
 1   Time       864368 non-null  int64 
 2   MicroSec   864368 non-null  int64 
 3   Type       864368 non-null  int64 
 4   Unit       864368 non-null  object
 5   Parameter  864368 non-null  object
 6   Message    864368 non-null  object
 7   Value1     411351 non-null  object
 8   Value2     495515 non-null  object
dtypes: int64(3), object(6)
memory usage: 65.9+ MB


In [136]:
log_rawDF.to_csv(file_name, index = False, sep='*')

In [141]:
log_rawDF = pd.read_csv(file_name, sep='*')

In [140]:
log_rawDF

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:0:13,1618916413,613129,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
1,2021-4-20 13:0:57,1618916457,9327,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
2,2021-4-20 13:1:38,1618916498,812633,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
3,2021-4-20 13:2:20,1618916540,214501,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
4,2021-4-20 13:3:3,1618916583,212528,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
...,...,...,...,...,...,...,...,...,...
864363,2022-4-20 13:0:24,1650452424,941081,1,FPC23_FPP23,Arduino,CO2 error value changed from 0.13 to 997,NaN,NaN
864364,2022-4-20 13:0:26,1650452426,155485,1,FPC23_AUT23,autoSyncMonitor,master of autoTemplateSync not found =&gt; no ...,NaN,NaN
864365,2022-4-20 13:0:26,1650452426,159772,1,FPC23_AUT23,autoSyncMonitor,Error!check autoTemplateSync template failed!,NaN,NaN
864366,2022-4-20 13:0:26,1650452426,164723,1,FPC23_AUT23,autoSyncMonitor,Timing exceeded warning level 11 &gt; 10,NaN,NaN


In [138]:
log_rawDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864368 entries, 0 to 864367
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Date_Time  864368 non-null  object
 1   Time       864368 non-null  int64 
 2   MicroSec   864368 non-null  int64 
 3   Type       864368 non-null  int64 
 4   Unit       864368 non-null  object
 5   Parameter  864368 non-null  object
 6   Message    864368 non-null  object
 7   Value1     411351 non-null  object
 8   Value2     495515 non-null  object
dtypes: int64(3), object(6)
memory usage: 59.4+ MB


## Inserting Units and Parameters into mariaDb tables

In [211]:
import sys
import mariadb

In [236]:
try: con = mariadb.connect( 
    user="root", 
    password="password", 
    host="localhost", 
    port=3306, 
    database="data_dashboard" 
)

except mariadb.Error as ex: 
    print(f"An error occurred while connecting to MariaDB: {ex}") 
    sys.exit(1) 

cur = con.cursor()

In [213]:
cur.execute("SELECT unit_id, unit_name FROM tbl_unit")

In [214]:
for (unit_id, unit_name) in cur:
    print("ID:", {unit_id}, "unit_name:", {unit_name})

ID: {2} unit_name: {'FFP'}
ID: {1} unit_name: {'FFU'}
ID: {3} unit_name: {'FPC11'}
ID: {4} unit_name: {'FPC12'}
ID: {5} unit_name: {'FPC13'}
ID: {6} unit_name: {'FPC14'}
ID: {7} unit_name: {'FPC21'}
ID: {8} unit_name: {'FPC22'}
ID: {9} unit_name: {'FPC23'}
ID: {10} unit_name: {'FPC24'}


In [217]:
# inserting list of parameters into DB table : tbl_parameters
query = 'INSERT INTO tbl_log_parameters (prm_id, prm_name) VALUES (%s, %s)'
tuples = pd.Series(log_parameter)
tuples = [(index+1, value) for index, value in tuples.items()]

In [218]:
tuples

[(1, 'Arduino'),
 (2, 'program'),
 (3, 'T61'),
 (4, 'water'),
 (5, 'T10'),
 (6, 'Reactor'),
 (7, 'T71'),
 (8, 'pneumatic'),
 (9, 'autoSyncMonitor'),
 (10, 'T42A'),
 (11, 'harvest'),
 (12, 'calculations'),
 (13, 'P1L'),
 (14, 'feeding'),
 (15, 'T64'),
 (16, 'T42'),
 (17, 'T53'),
 (18, 'T51'),
 (19, 'T73'),
 (20, 'T74'),
 (21, 'T65'),
 (22, 'T63'),
 (23, 'T72'),
 (24, 'T52'),
 (25, 'T54'),
 (26, 'PT1_V4'),
 (27, 'T55'),
 (28, 'T43'),
 (29, 'LEDlighting'),
 (30, 'RM29'),
 (31, 'RM6'),
 (32, 'V74'),
 (33, 'RM12'),
 (34, 'RM13'),
 (35, 'RM14'),
 (36, 'RM15'),
 (37, 'RM16'),
 (38, 'RM17'),
 (39, 'RM19'),
 (40, 'RM2'),
 (41, 'RM21'),
 (42, 'RM22'),
 (43, 'RM23'),
 (44, 'T41A'),
 (45, 'T43A'),
 (46, 'RM35'),
 (47, 'RM4'),
 (48, 'RM5'),
 (49, 'RM7'),
 (50, 'P42'),
 (51, 'RM1'),
 (52, 'RM30'),
 (53, 'Arduino_QG'),
 (54, 'Arduino_pH'),
 (55, 'V3'),
 (56, 'P1R'),
 (57, 'P2R'),
 (58, 'CO2_V2'),
 (59, 'R1'),
 (60, 'PTdiff_K2'),
 (61, 'L1W'),
 (62, 'Arduino_TT1'),
 (63, 'Arduino_LT1'),
 (64, 'Arduino

In [219]:
cur.executemany(query, tuples)

In [221]:
cur.execute("SELECT prm_id, prm_name FROM tbl_log_parameters")

In [222]:
for (prm_id, prm_name) in cur:
    print("ID:", {prm_id}, "unit_name:", {prm_name})

ID: {1} unit_name: {'Arduino'}
ID: {101} unit_name: {'Arduino1'}
ID: {66} unit_name: {'Arduino1_LT41'}
ID: {125} unit_name: {'Arduino1_LT42'}
ID: {126} unit_name: {'Arduino1_LT43'}
ID: {119} unit_name: {'Arduino1_LT45'}
ID: {127} unit_name: {'Arduino1_LT51'}
ID: {64} unit_name: {'Arduino1_LT52'}
ID: {115} unit_name: {'Arduino1_LT53'}
ID: {118} unit_name: {'Arduino1_LT54'}
ID: {121} unit_name: {'Arduino1_LT55'}
ID: {160} unit_name: {'Arduino1_PT10'}
ID: {161} unit_name: {'Arduino1_PT11'}
ID: {129} unit_name: {'Arduino2'}
ID: {122} unit_name: {'Arduino2_LT10'}
ID: {67} unit_name: {'Arduino2_LT61'}
ID: {128} unit_name: {'Arduino2_LT62'}
ID: {113} unit_name: {'Arduino2_LT63'}
ID: {116} unit_name: {'Arduino2_LT64'}
ID: {114} unit_name: {'Arduino2_LT65'}
ID: {123} unit_name: {'Arduino2_LT71'}
ID: {65} unit_name: {'Arduino2_LT72'}
ID: {111} unit_name: {'Arduino2_LT73'}
ID: {120} unit_name: {'Arduino2_LT74'}
ID: {117} unit_name: {'Arduino2_LT75'}
ID: {162} unit_name: {'Arduino2_noInUse'}
ID: {

In [223]:
con.commit() 

In [231]:
# inserting list of units into DB table : tbl_unit
query = 'INSERT INTO tbl_unit (unit_id, unit_name) VALUES (%s, %s)'
tuples = pd.Series(log_unit)
tuples = [(index+1, value) for index, value in tuples.items()]
tuples

[(1, 'FPC11_FPP11'),
 (2, 'FPC14'),
 (3, 'FFU'),
 (4, 'FPC22'),
 (5, 'FPC21_AUT21'),
 (6, 'FPC21'),
 (7, 'FPC23'),
 (8, 'FPC13'),
 (9, 'FPC24'),
 (10, 'FPC31'),
 (11, 'FPC23_FPP23'),
 (12, 'FPC41'),
 (13, 'FPC42'),
 (14, 'FPC11'),
 (15, 'Product'),
 (16, 'FPC21_FPP21'),
 (17, 'FPC13_FPP13'),
 (18, 'FPC24_FPP24'),
 (19, 'FFP_FFP'),
 (20, 'FPC31_FPP31'),
 (21, 'FPC22_FPP22'),
 (22, 'FPC12'),
 (23, 'FPP11'),
 (24, 'FPC14_FPP14'),
 (25, 'FPP21'),
 (26, 'FPP13'),
 (27, 'FPP14'),
 (28, 'FPP'),
 (29, 'FPC12_FPP'),
 (30, 'FPP31'),
 (31, 'FFP_AUTFU'),
 (32, 'FPC13_AUT13'),
 (33, 'FPC22_AUT22'),
 (34, 'FPC11_AUT11'),
 (35, 'FPC12_AUT'),
 (36, 'FPC31_AUT31'),
 (37, 'FPC24_AUT24'),
 (38, 'FPC23_AUT23'),
 (39, 'scheduler'),
 (40, 'FPC14_AUT14')]

In [232]:
cur.executemany(query, tuples)
con.commit() 

In [224]:
con.close()